In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.7 MB/s eta 0:00:00


In [ ]:
from tabulate import tabulate
from nltk.translate.bleu_score import sentence_bleu
from tqdm.auto import tqdm
import torch
import os

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import string
import random
from tqdm.auto import tqdm
import torch
import transformers
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from transformers import AdamW, get_cosine_schedule_with_warmup
from transformers import pipeline, set_seed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
folder_path = '/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning'
for filename in os.listdir(folder_path):
    print(filename)

Finetuned_GPT2_10_epochs_V0.pt
FineTune_gpt2_engine_type_5_epochs.zip
FineTune_gpt2_drive_type_5_epochs.zip
FineTune_gpt2_question_answering_5_epochs.zip
FineTune_gpt2_question_answering_engine_type_And_Drive_Type.zip
FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip
FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.zip
FineTune_gpt2_Distilled_version_10_epochs.zip
FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.zip
FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.zip


In [ ]:
import torch
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define the path to your GPT-2 model file
model_path_1 = '/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/Finetuned_GPT2_10_epochs_V0.pt'

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.zip" -d FineTune_gpt2_engine_type_And_Drive_Type_prunned_10
FineTune_gpt2_engine_type_And_Drive_Type_prunned_10 = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10')
FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.to(device)


Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.zip
  inflating: FineTune_gpt2_engine_type_And_Drive_Type_prunned_10/generation_config.json  
  inflating: FineTune_gpt2_engine_type_And_Drive_Type_prunned_10/config.json  
  inflating: FineTune_gpt2_engine_type_And_Drive_Type_prunned_10/pytorch_model.bin  


Some weights of the model checkpoint at /content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10 were not used when initializing GPT2LMHeadModel: ['transformer.h.4.attn.c_proj.weight_orig', 'transformer.h.5.attn.c_attn.weight_orig', 'transformer.h.3.mlp.c_proj.weight_orig', 'transformer.h.5.ln_2.weight_mask', 'transformer.h.6.mlp.c_fc.weight_mask', 'transformer.h.8.attn.c_proj.weight_orig', 'transformer.h.1.ln_2.weight_mask', 'transformer.h.8.ln_1.weight_mask', 'transformer.h.3.ln_2.weight_orig', 'transformer.h.7.ln_2.weight_mask', 'transformer.h.4.ln_2.weight_mask', 'transformer.h.3.mlp.c_fc.weight_mask', 'transformer.h.10.ln_2.weight_orig', 'transformer.h.1.attn.c_attn.weight_orig', 'transformer.h.2.ln_1.weight_orig', 'transformer.h.1.attn.c_proj.weight_mask', 'transformer.h.5.mlp.c_fc.weight_orig', 'transformer.h.3.attn.c_attn.weight_mask', 'transformer.h.3.ln_1.weight_orig', 'transformer.h.11.mlp.c_proj.weight_orig', 'transformer.h.3.attn.c_proj.weight_mask', 'transformer.h.11.

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.zip" -d FineTune_gpt2_engine_type_And_Drive_Type_prunned_50
FineTune_gpt2_engine_type_And_Drive_Type_prunned_50 = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50')
FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.to(device)


Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.zip
  inflating: FineTune_gpt2_engine_type_And_Drive_Type_prunned_50/pytorch_model.bin  
  inflating: FineTune_gpt2_engine_type_And_Drive_Type_prunned_50/generation_config.json  
  inflating: FineTune_gpt2_engine_type_And_Drive_Type_prunned_50/config.json  


Some weights of the model checkpoint at /content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50 were not used when initializing GPT2LMHeadModel: ['transformer.h.4.attn.c_proj.weight_orig', 'transformer.h.5.attn.c_attn.weight_orig', 'transformer.h.3.mlp.c_proj.weight_orig', 'transformer.h.5.ln_2.weight_mask', 'transformer.h.6.mlp.c_fc.weight_mask', 'transformer.h.8.attn.c_proj.weight_orig', 'transformer.h.1.ln_2.weight_mask', 'transformer.h.8.ln_1.weight_mask', 'transformer.h.3.ln_2.weight_orig', 'transformer.h.7.ln_2.weight_mask', 'transformer.h.4.ln_2.weight_mask', 'transformer.h.3.mlp.c_fc.weight_mask', 'transformer.h.10.ln_2.weight_orig', 'transformer.h.1.attn.c_attn.weight_orig', 'transformer.h.2.ln_1.weight_orig', 'transformer.h.1.attn.c_proj.weight_mask', 'transformer.h.5.mlp.c_fc.weight_orig', 'transformer.h.3.attn.c_attn.weight_mask', 'transformer.h.3.ln_1.weight_orig', 'transformer.h.11.mlp.c_proj.weight_orig', 'transformer.h.3.attn.c_proj.weight_mask', 'transformer.h.11.

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_Distilled_version_10_epochs.zip" -d FineTune_gpt2_Distilled_version_10_epochs



NotImplementedError: ignored

In [ ]:
Model_FineTune_gpt2_Distilled_version_10_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_Distilled_version_10_epochs')
Model_FineTune_gpt2_Distilled_version_10_epochs.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

model_dist = AutoModelForCausalLM.from_pretrained("distilgpt2")
model_dist.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
print(device)

cuda


In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip" -d model_FineTune_Engine_type

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip
  inflating: model_FineTune_Engine_type/pytorch_model.bin  
  inflating: model_FineTune_Engine_type/generation_config.json  
  inflating: model_FineTune_Engine_type/config.json  


In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip" -d model_FineTune_Drive_type

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip
  inflating: model_FineTune_Drive_type/pytorch_model.bin  
  inflating: model_FineTune_Drive_type/generation_config.json  
  inflating: model_FineTune_Drive_type/config.json  


In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip" -d FineTune_gpt2_drive_type_And_Engine_type_5_epochs
Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_drive_type_And_Engine_type_5_epochs')
Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs/config.json  
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs/pytorch_model.bin  
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs/generation_config.json  


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.zip" -d FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS
FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS')
FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.zip
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS/config.json  
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS/pytorch_model.bin  
  inflating: FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS/generation_config.json  


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_engine_type_And_Drive_Type.zip" -d FineTune_gpt2_question_answering_engine_type_And_Drive_Type
FineTune_gpt2_question_answering_engine_type_And_Drive_Type = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_question_answering_engine_type_And_Drive_Type')
FineTune_gpt2_question_answering_engine_type_And_Drive_Type.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_engine_type_And_Drive_Type.zip
  inflating: FineTune_gpt2_question_answering_engine_type_And_Drive_Type/config.json  
  inflating: FineTune_gpt2_question_answering_engine_type_And_Drive_Type/generation_config.json  
  inflating: FineTune_gpt2_question_answering_engine_type_And_Drive_Type/pytorch_model.bin  


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_5_epochs.zip" -d FineTune_gpt2_question_answering_5_epochs
model_FineTune_gpt2_question_answering_5_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_question_answering_5_epochs')
model_FineTune_gpt2_question_answering_5_epochs.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_5_epochs.zip
  inflating: FineTune_gpt2_question_answering_5_epochs/config.json  
  inflating: FineTune_gpt2_question_answering_5_epochs/pytorch_model.bin  
  inflating: FineTune_gpt2_question_answering_5_epochs/generation_config.json  


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model_FineTune_Engine_type = GPT2LMHeadModel.from_pretrained('/content/model_FineTune_Engine_type')
model_FineTune_Engine_type.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model_FineTune_Drive_type = GPT2LMHeadModel.from_pretrained('/content/model_FineTune_Drive_type')
model_FineTune_Drive_type.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import logging
# Set the logging level to ERROR for the transformers library
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
model_loaded = GPT2LMHeadModel.from_pretrained('gpt2')
model_loaded.to(device)
model_loaded.load_state_dict(torch.load(model_path_1))

<All keys matched successfully>

In [ ]:
def calculate_bleu(model, tokenizer, data):
    bleu_scores = []
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Calculating BLEU score"):
        # Get the reference text from the 'prompt_target' column
        reference_text = row['prompt_target']
        # Generate the text using the model
        prompt = f"Car: {row['Full Name']}\ndescription:"
        generated_text = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)
        output_text = model.generate(
            generated_text,
            max_length=250,
            temperature=0.005,
            do_sample=True,
            top_k=100,
            top_p=0.95,
            repetition_penalty=1.2,
            num_return_sequences=1,
        )
        decoded_output = tokenizer.decode(output_text[0], skip_special_tokens=True)
        # Calculate the BLEU score
        score = sentence_bleu([reference_text], decoded_output)
        bleu_scores.append(score)
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu_score

In [ ]:
def evaluate_models(models, tokenizer, data):
    results = []
    for model_name, model in tqdm(models.items(), desc="Evaluating models"):
        avg_bleu_score = calculate_bleu(model, tokenizer, data)
        results.append((model_name, avg_bleu_score))

    # Print the results in a table format
    headers = ['Model', 'BLEU Score']
    table = tabulate(results, headers=headers, tablefmt='orgtbl')
    print(table)

In [ ]:
models = {
    'Distilled_GPT2': model_dist,
    'FineTune_gpt2_Distilled_version_10_epochs': Model_FineTune_gpt2_Distilled_version_10_epochs
}

In [ ]:
models = {
    'model GPT2': model,
    'model_finetune_10_epochs': model_loaded,
    'model_FineTune with_prefinetune_type' : model_FineTune_Engine_type,
    'model_FineTune_with prefinetune_drive_type' : model_FineTune_Drive_type,
    'Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs':Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs,
    'model_FineTune_gpt2_question_answering_5_epochs':model_FineTune_gpt2_question_answering_5_epochs,
    'FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS': FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS,
    'FineTune_gpt2_question_answering_engine_type_And_Drive_Type': FineTune_gpt2_question_answering_engine_type_And_Drive_Type
}

In [ ]:
models = {
    'FineTune_gpt2_engine_type_And_Drive_Type_prunned_50': FineTune_gpt2_engine_type_And_Drive_Type_prunned_50,
    'FineTune_gpt2_engine_type_And_Drive_Type_prunned_10': FineTune_gpt2_engine_type_And_Drive_Type_prunned_10 }

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Data/Summary_Final_context.csv")

In [ ]:
data= data.drop(columns=['Pros','Cons','horsepower','engine_type','Torque','Torque_RPM','drive_type','transmission','width','Length','Height'])

In [ ]:
prompts = []
targets = []
li = []
for index, row in data.iterrows():
    carName = str(row['Full Name']).replace('\n', ' ')
    description = str(row['final_context']).replace('\n', ' ')
    prompt = 'Car: ' + carName+'\nDescription: '
    target = description
    targets.append(target)
    prompts.append(prompt)
    li.append(prompt + ' ' + target)
data['prompt_target'] = li

In [ ]:
subset_data = data.sample(frac=0.12, random_state=42)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
#evaluate_models(models, tokenizer, subset_data)

Evaluating models:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

| Model                                                                  |   BLEU Score |
|------------------------------------------------------------------------+--------------|
| model GPT2                                                             |    0.0656258 |
| model_finetune_10_epochs                                               |    0.282152  |
| model_FineTune with_prefinetune_type                                   |    0.26483   |
| model_FineTune_with prefinetune_drive_type                             |    0.264649  |
| Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs                |    0.300696  |
| model_FineTune_gpt2_question_answering_5_epochs                        |    0.263833  |
| FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS |    0.182263  |
| FineTune_gpt2_question_answering_engine_type_And_Drive_Type            |    0.258602  |


In [ ]:
#evaluate_models(models, tokenizer, subset_data)

Evaluating models:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

| Model                                     |   BLEU Score |
|-------------------------------------------+--------------|
| Distilled_GPT2                            |    0.0523098 |
| FineTune_gpt2_Distilled_version_10_epochs |    0.20256   |


In [ ]:
evaluate_models(models, tokenizer, subset_data)

Evaluating models:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

Calculating BLEU score:   0%|          | 0/377 [00:00<?, ?it/s]

| Model                                               |   BLEU Score |
|-----------------------------------------------------+--------------|
| FineTune_gpt2_engine_type_And_Drive_Type_prunned_50 |    0.0648703 |
| FineTune_gpt2_engine_type_And_Drive_Type_prunned_10 |    0.0514739 |
